<a href="https://colab.research.google.com/github/Benzenoid44/CSI-/blob/main/final_drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import os
from PIL import Image
from tqdm import tqdm   # ✅ for progress bars

# === 1️⃣ Path to dataset ===
data_dir = "/content/drive/MyDrive/Crop_Project/apple_dataset"

# === 2️⃣ Transformations ===
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# === 3️⃣ Load dataset ===
dataset = datasets.ImageFolder(data_dir, transform=train_transform)
print("✅ Classes:", dataset.classes)
print("Total images:", len(dataset))

# === 4️⃣ Split into train/val ===
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Apply correct transforms for each split
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# === 5️⃣ Define model (ResNet18) ===
model = models.resnet18(weights='IMAGENET1K_V1')
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# === 6️⃣ Loss + Optimizer ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === 7️⃣ Training with accuracy & tqdm progress ===
epochs = 5
for epoch in range(epochs):
    print(f"\n🔹 Epoch {epoch+1}/{epochs}")
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    # Training loop with progress bar
    for imgs, labels in tqdm(train_loader, desc="Training", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc="Validating", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

print("\n✅ Training complete!")

# === 8️⃣ Save model ===
torch.save(model.state_dict(), "/content/drive/MyDrive/Crop_Project/apple_model_trained.pth")
print("✅ Model saved!")

# === 9️⃣ Test prediction ===
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = val_transform(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)
    return dataset.classes[pred.item()]

# Example test
test_img = "/content/drive/MyDrive/Crop_Project/apple_dataset/Apple___Apple_scab/84ba6fe9-427e-415b-8bfe-a1f8670e1537___FREC_Scab 3530.JPG"
print("Predicted disease:", predict_image(test_img))


✅ Classes: ['Apple___Apple_scab', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Apple_black_rot']
Total images: 7771
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 96.9MB/s]



🔹 Epoch 1/5


Loss: 0.1337 | Train Acc: 95.64% | Val Loss: 0.0247 | Val Acc: 99.16%

🔹 Epoch 2/5


KeyboardInterrupt: 

In [3]:
# Save partially trained model after interrupt
torch.save(model.state_dict(), "/content/drive/MyDrive/Crop_Project/apple_model_partial_epoch1.pth")
print("✅ Model saved after partial training!")


✅ Model saved after partial training!


In [4]:
import torch
from torchvision import models, transforms
from PIL import Image

# Path to saved model
model_path = "/content/drive/MyDrive/Crop_Project/apple_model_partial_epoch1.pth"
data_dir = "/content/drive/MyDrive/Crop_Project/apple_dataset"

# Same transform used during training
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Get class names (must match your dataset folder structure)
from torchvision import datasets
dataset = datasets.ImageFolder(data_dir, transform=val_transform)
classes = dataset.classes
print("✅ Loaded classes:", classes)

# Rebuild the model architecture
model = models.resnet18(weights=None)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, len(classes))

# Load the trained weights
model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("✅ Model loaded and ready for testing!")


✅ Loaded classes: ['Apple___Apple_scab', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Apple_black_rot']
✅ Model loaded and ready for testing!


In [5]:
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = val_transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(img)
        _, pred = torch.max(outputs, 1)
        predicted_class = classes[pred.item()]

    return predicted_class


In [7]:
test_img = "/content/drive/MyDrive/Crop_Project/aata_nako_train/apple_dataset/Apple___healthy/0055dd26-23a7-4415-ac61-e0b44ebfaf80___RS_HL 5672.JPG"

prediction = predict_image(test_img)
print(f"🍏 Predicted Disease: {prediction}")


🍏 Predicted Disease: Apple___healthy
